In [1]:
#librerias estandar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [23]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv", index_col="ID")
#data.drop(columns=["F_TIENEINTERNET.1"], inplace=True)  
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


**Nota aclaratoria**

El preprocesado con promedios hace referencia a la estrategia de reducción de categorias de la variable PRGM_ACADEMICO. En este caso la reducción se hizo abarcando en la misma categoria aquellos programas cuyos promedios fueran similares. Esto con la intención de lograr una reducción de dimensionalidad mayor al método del umbral

In [24]:
#preprocesado

#tratamiento de variable objetivo
orden = ["bajo", "medio-bajo", "medio-alto", "alto"]
data['RENDIMIENTO_GLOBAL'] = pd.Categorical(data['RENDIMIENTO_GLOBAL'], ordered=True, categories=orden)
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].cat.codes  # convierte a 0,1,2,3

#Nulos
#True si el registro no tiene ningun valor nulo, False en caso contrario
data["DATOS_COMPLETOS"] = data.isna().sum(axis=1) == 0
data = data.fillna("sin_dato") #rellenar nulos

#columna repetida
data = data.drop(columns=["F_TIENEINTERNET.1"])

#reducción de categorias de E_PRGM_ACADEMICO usando promedios de rendimiento

#media del rendimiento por programa
media_rend = data.groupby('E_PRGM_ACADEMICO')['RENDIMIENTO_GLOBAL'].mean()
#40 grupos ordenados por rendimiento
clusters = pd.qcut(media_rend, q=50, labels=False)
programa_to_cluster = clusters.to_dict() #para luego aplicar al test.csv
# clusters
data['E_PRGM_ACADEMICO_AGRUPADO'] = data['E_PRGM_ACADEMICO'].map(clusters)
data.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)


#binarias
binarias = [
    "F_TIENEINTERNET",
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
    "E_PRIVADO_LIBERTAD"
]

for col in binarias:
    data[col] = (
        data[col]
        .astype(str)
        .str.strip()
        .str.upper()
        .replace({
            "SI": 1, "S": 1,
            "NO": 0, "N": 0,
            "SIN_DATO": -1
        })
    )

#ordinales
# Codificación para E_VALORMATRICULAUNIVERSIDAD
map_matricula = {
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7,
    "No pagó matrícula": 0,
    "sin_dato": -1
}
data["E_VALORMATRICULAUNIVERSIDAD"] = data["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)

# Codificación para F_ESTRATOVIVIENDA
map_estrato = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": 0,
    "sin_dato": -1
}
data["F_ESTRATOVIVIENDA"] = data["F_ESTRATOVIVIENDA"].map(map_estrato)

# Codificación para E_HORASSEMANATRABAJA
map_horas = {
    "0": 0,   # por si aparece como string
    0: 0,     # por si aparece como número
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4,
    "sin_dato": -1
}
data["E_HORASSEMANATRABAJA"] = data["E_HORASSEMANATRABAJA"].map(map_horas)

educacion_map = {
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    
    # categorías sin información → -1
    "sin_dato": -1,
    "No sabe": -1,
    "No Aplica": -1
}

data["F_EDUCACIONPADRE"] = data["F_EDUCACIONPADRE"].map(educacion_map)
data["F_EDUCACIONMADRE"] = data["F_EDUCACIONMADRE"].map(educacion_map)

#one hot
data = pd.get_dummies(data, columns=["E_PRGM_DEPARTAMENTO"], prefix="DEP", drop_first=False)
data = pd.get_dummies(data, columns=["E_PRGM_ACADEMICO_AGRUPADO"], prefix="PRGM", drop_first=False)

#periodo academico
data["PERIODO_ACADEMICO"] = data["PERIODO_ACADEMICO"].astype(str)

data["ANIO"] = data["PERIODO_ACADEMICO"].str[:4].astype(int)
data["PERIODO"] = data["PERIODO_ACADEMICO"].str[-1].astype(int)

data.drop(columns=["PERIODO_ACADEMICO"], inplace=True)

#numericas
cols_norm = ["INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]


scaler = StandardScaler()
data[cols_norm] = scaler.fit_transform(data[cols_norm])

#data.drop(columns=["E_PRGM_ACADEMICO_AGRUPADO"], inplace=True)
#data.drop(columns=["E_PRGM_DEPARTAMENTO"], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_11140\4040566030.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_11140\4040566030.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_11140\4040566030.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set

In [25]:
data.head()

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,PRGM_42,PRGM_43,PRGM_44,PRGM_45,PRGM_46,PRGM_47,PRGM_48,PRGM_49,ANIO,PERIODO
ID,,,,,,,,,,,,,,,,,,,,,
904256,6,1,3,1,5,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
645256,4,0,3,0,6,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
308367,4,4,3,1,4,1,0,0,0,0,...,False,False,False,False,False,False,False,False,2020,3
470353,5,0,4,1,-1,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2019,5
989032,4,3,3,1,2,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier, Pool
#modelo con mejores parametros

y = data["RENDIMIENTO_GLOBAL"]                 # variable a predecir
X = data.drop(columns=["RENDIMIENTO_GLOBAL"])  # todo lo demás


X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

#modelo encontrado
best_params = {
    'random_strength': 2,
    'learning_rate': 0.01,
    'l2_leaf_reg': 1,
    'iterations': 5000,
    'depth': 10,
    'border_count': 64,
    'bagging_temperature': 0.1,
    'auto_class_weights': 'Balanced'
}

model = CatBoostClassifier(
    **best_params,
    task_type="GPU",         
    loss_function="MultiClass",
    eval_metric="Accuracy",
    random_seed=42,
    verbose=200
)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    
)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))



0:	learn: 0.3783976	test: 0.3744987	best: 0.3744987 (0)	total: 59.4ms	remaining: 4m 57s
200:	learn: 0.4162688	test: 0.4111759	best: 0.4111759 (200)	total: 8.51s	remaining: 3m 23s
400:	learn: 0.4267162	test: 0.4208512	best: 0.4208512 (400)	total: 15.9s	remaining: 3m 1s
600:	learn: 0.4323909	test: 0.4261380	best: 0.4261380 (600)	total: 23s	remaining: 2m 48s
800:	learn: 0.4366148	test: 0.4289658	best: 0.4289658 (800)	total: 30.2s	remaining: 2m 38s
1000:	learn: 0.4401544	test: 0.4305593	best: 0.4306323 (988)	total: 37.4s	remaining: 2m 29s
1200:	learn: 0.4436460	test: 0.4323691	best: 0.4324843 (1180)	total: 44.7s	remaining: 2m 21s
1400:	learn: 0.4472453	test: 0.4336745	best: 0.4338415 (1386)	total: 52.5s	remaining: 2m 14s
1600:	learn: 0.4504034	test: 0.4348895	best: 0.4348895 (1600)	total: 1m	remaining: 2m 7s
1800:	learn: 0.4537414	test: 0.4355171	best: 0.4356185 (1776)	total: 1m 8s	remaining: 2m 1s
2000:	learn: 0.4569664	test: 0.4359010	best: 0.4360090 (1954)	total: 1m 16s	remaining: 1m 54